# Groq + Box MCP: Real-Time, Fast Document Intelligence
This notebook is designed for Python developers who want to securely enrich Groq's fast inference with private context retrieved via [Box's MCP Server](https://developer.box.com/guides/box-mcp/) functionality.

We will achieve this through three simple steps:
1. Set up **Groq client** for fast inference.
2. Connect to the **Remote Box MCP Server** to securely search through private content in Box.
3. Seemlessly **connect the Groq client to the remove MCP server** through the Responses API.


---

## Prerequisites

This cookbook assumes you have:
1. A Box **developer** account. If you don't have one, you can sign up for a free account at [box](https://account.box.com/signup/n/developer).
2. If you want to run this notebook from Google Colab or other, skip the following. If instead you want to run this locally, make sure you have Python >3.10 installed (3.12 recommended). Create a virtual environment with `python -m venv .venv`. Activate this virtual env as `source .venv/bin/activate`
3. Run `pip install -r requirements.txt` from the `mcp-box` folder in `tutorials/`.
4. If you are running this locally, also run `pip install jupyter ipykernel`


## Getting Started

If you've gone through the pre-requisites, follow these steps to get set up:
1. **Create a Platform App** in the [Box Developer Console](https://app.box.com/developers/console) and enable it. (NOTE - if you have a developer account, you should see "Box DEVELOPER" in the top-left). The following process is a little tedious but it's only a one-time thing.

    1. Select `Custom App`.
    2. Enter an app name. For purpose, select `Automation`.
    3. For authentication method, select the option called `Server Authentication (Client Credentials Grant)
    4. Click on the created platform app. This is where you will later be able to fetch the `Client ID`, `Client Secret`, and configure anything for your app.
    5. From this menu, select the `Configuration` tab. In `App Access Level`, select `App + Enterprise Access`. Save changes.
    6. Click on the `Authorization` tab. Here click on `Review and Submit`. 
    7. You'll now have to head to the `Admin Console` to authorize the app. This [link](https://app.box.com/master/platform-apps) should take you directly there. It should say `Pending Authorization`. Go ahead and authorize it.
    8. Go back to the dev console, and open the app you just created. This [link](https://app.box.com/developers/console) should list the app.
    9. From the app's general settings tab, copy the `Enterprise ID`. From the configuration tab, copy the `Client ID` and `Client Secret` (might require you to click fetch). Use these values to set environment values for all of these before running the notebook.

```
export BOX_CLIENT_ID=<value>
export BOX_CLIENT_SECRET=<value>
export BOX_ENTERPRISE_ID=<value>
```

2. **Sign up** for Groq at [console.groq.com](https://console.groq.com/keys) to get your free API key. Once you have the API key set the environment variable `GROQ_API_KEY` with its value.

```
export GROQ_API_KEY=<value>
```


In [ ]:
import os
from box_sdk_gen import (
    BoxClient,
    BoxCCGAuth,
    CCGConfig,
)

BOX_CLIENT_ID = os.getenv("BOX_CLIENT_ID", "<value>")
BOX_CLIENT_SECRET = os.getenv("BOX_CLIENT_SECRET", "<value>")
BOX_ENTERPRISE_ID = os.getenv("BOX_ENTERPRISE_ID", "<value>")

ccg_config = CCGConfig(
    client_id=BOX_CLIENT_ID,
    client_secret=BOX_CLIENT_SECRET,
    enterprise_id=BOX_ENTERPRISE_ID,
)
auth = BoxCCGAuth(config=ccg_config)
box_client = BoxClient(auth=auth)

BOX_MCP_TOKEN = box_client.auth.retrieve_token().access_token
# To run locally you can also just set the token directly 
# this can be generated from the developer console
# comment out the line above and uncomment the line below
# BOX_MCP_TOKEN = "<value>"

In [ ]:
import json
import os
import time
from datetime import datetime

# Groq API Configuration
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Check if API key is set
if not GROQ_API_KEY:
    print("Please set your Groq production API key")
else:
    print("Groq API key configured successfully!")
if not BOX_MCP_TOKEN:
    print("Please set your Box Developer Token")
else:
    print("Box Developer Token configured successfully!")

Select the foundation model to power inference. Let's try OpenAI's flagship open-weights MoE model, [gpt-oss-120b](https://console.groq.com/docs/model/openai/gpt-oss-120b), available via Groq for fast inference.

In [ ]:
# Model configuration
MODEL = "openai/gpt-oss-120b"

We will upload all files that are stored in the `data` folder by default. If you'd like to try out with your own data, please add some data there.

In [ ]:
DATA_FOLDER = "data"
BOX_PARENT_FOLDER_NAME = "groq-cookbook"

In [ ]:
files = os.listdir(DATA_FOLDER)
if not files:
    raise ValueError(f"No files to upload. Please add some data to the {DATA_FOLDER} folder.")
print("Files to be uploaded:")
print("-" * 50)
files_to_upload = []
for f in files:
    print(f)
    files_to_upload.append((os.path.join(DATA_FOLDER, f), f))

We will create a folder in our Box MCP

In [ ]:
from box_sdk_gen import CreateFolderParent

ROOT_FOLDER_ID = "0"

print(f"Creating folder: {BOX_PARENT_FOLDER_NAME} in Box")
folder = box_client.folders.create_folder(BOX_PARENT_FOLDER_NAME, CreateFolderParent(id=ROOT_FOLDER_ID))

parent_folder_id = folder.id
parent_folder_name = folder.name
print(f"Created folder with ID: {parent_folder_id}")

These are helper functions that you may find useful. You don't need to run them.

In [ ]:
def delete_folder(client, folder_id, recursive=True):
    """
    Delete the folder with folder_id and all of its contents.
    Be careful with this function, it will delete the folder and all of its contents.
    """
    try:
        client.folders.delete_folder_by_id(folder_id, recursive=recursive)
        print(f"Deleted folder {folder_id} {'recursively' if recursive else ''} via direct API call")
        return
    except Exception as e:
        raise

def list_folders(client, parent_folder_id):
    folder_ids = []
    items = client.folders.get_folder_items(parent_folder_id)
    for entry in items.entries:
        if entry.type == 'folder':
            folder_ids.append((entry.id, entry.name))
    return folder_ids
    
def list_files(client, parent_folder_id, recursive=False):
    file_ids = []
    items = client.folders.get_folder_items(parent_folder_id)
    for entry in items.entries:
        if entry.type == 'file':
            file_ids.append((entry.id, entry.name))
        elif entry.type == 'folder' and recursive:
            file_ids.extend(list_files(client, entry.id, recursive=True))
    return file_ids

folder_ids = list_folders(box_client, ROOT_FOLDER_ID)
print(f"Found {len(folder_ids)} folders")
print("-" * 50)
for folder_id, folder_name in folder_ids:
    print(f"{folder_id} - {folder_name}")

file_ids = list_files(box_client, ROOT_FOLDER_ID, recursive=True)
print(f"Found {len(file_ids)} files")
print("-" * 50)
for file_id, file_name in file_ids:
    print(f"{file_id} - {file_name}")



In [ ]:
from box_sdk_gen import UploadFileAttributes, UploadFileAttributesParentField

def upload_data_to_box(client, files, parent_folder_id):
    assert isinstance(files, list)
    assert isinstance(files[0], tuple), "files must be a list of tuples (file_path, file_name)"
    successful_uploads = []
    for file_path, file_name in files:
        try:
            with open(file_path, "rb") as file:
                _ = client.uploads.upload_file(
                    UploadFileAttributes(
                        name=file_name, parent=UploadFileAttributesParentField(id=parent_folder_id)
                    ),
                    file,
            )
            successful_uploads.append(file_name)
        except Exception as e:
            print(f"Error uploading file {file_name}: {e}")
            pass
    print("Successfully uploaded:")
    print("-" * 50)
    for f in successful_uploads:
        print(f)


## Pre-requisite 1: Upload data to Box
Keep in mind, as mentioned before, we'll upload all files stored in the `data` local folder by default. We have a single sample file there to power this demo. If you'd like to try out with your own data, please add some data there.

In [ ]:
upload_data_to_box(box_client, files_to_upload, parent_folder_id)

## Step 1: Set up the Groq client

In [ ]:
from openai import OpenAI

# set up Groq client
client = OpenAI(base_url="https://api.groq.com/api/openai/v1", api_key=GROQ_API_KEY)

## Step 2: Set up Box's remote MCP server

In [ ]:
# set up Box MCP server
tools = [
    {
        "type": "mcp",
        "server_url": f"https://mcp.box.com",
        "server_label": "box",
        "require_approval": "never",
        "headers": { "Authorization": f"Bearer {BOX_MCP_TOKEN}" }
    }
]

## Step 3: Connect Groq to the Box MCP through Groq's OpenAI-compatible Responses API

In [ ]:
def connect_groq_to_box(client, tools, query):
    """
    Connect Groq client to Box server through responses API.

    This function demonstrates the speed and accuracy of combining:
    - Groq's fast LLM inference (500+ tokens/second)
    - Box MCP server for real-time document search and retrieval
    """

    start_time = time.time()

    # Call Groq with Box MCP integration using responses API
    response = client.responses.create(
        model=MODEL,
        input=query,
        tools=tools,
        stream=False,
        temperature=0.1,
    )

    total_time = time.time() - start_time

    # Get response content from responses API
    content = (
        response.output_text if hasattr(response, "output_text") else str(response)
    )

    # collect executed tools (MCP tool calls)
    executed_tools = []

    # Extract MCP calls from response if available
    if hasattr(response, "output") and response.output:
        for output_item in response.output:
            if hasattr(output_item, "type") and output_item.type == "mcp_call":
                executed_tools.append(
                    {
                        "type": "mcp",
                        "arguments": getattr(output_item, "arguments", "{}"),
                        "output": getattr(output_item, "output", ""),
                        "name": getattr(output_item, "name", ""),
                        "server_label": getattr(output_item, "server_label", ""),
                    }
                )
    return {
        "content": content,
        "total_time": total_time,
        "mcp_calls_performed": executed_tools,
        "timestamp": datetime.now().isoformat(),
    }

Let's implement a helper function to display MCP tool calls and their results. This will provide transparency into which tools were called, their arguments, and outputs.

In [ ]:
def print_mcp_calls(mcp_calls):
    executed_tools = mcp_calls["mcp_calls_performed"]
    if executed_tools:
        print(f"\nBOX MCP CALLS: Found {len(executed_tools)} tool call(s):")
        print("-" * 50)
        for i, tool in enumerate(executed_tools, 1):
            print(f"\nTool Call #{i}")
            print(f"   Type: {tool['type']}")
            print(f"   Tool Name: {tool['name']}")
            print(f"   Server: {tool['server_label']}")
            try:
                if tool["arguments"]:
                    args = (
                        json.loads(tool["arguments"])
                        if isinstance(tool["arguments"], str)
                        else tool["arguments"]
                    )
                    print(f"   Arguments: {args}")

                # Print model results for transparency
                if tool["output"]:
                    output_data = (
                        json.loads(tool["output"])
                        if isinstance(tool["output"], str)
                        else tool["output"]
                    )
                    if isinstance(output_data, dict) and "models" in output_data:
                        print(f"   Models found: {len(output_data['models'])}")
                        for j, model in enumerate(
                            output_data["models"][:5], 1
                        ):  # Show top 5
                            model_name = model.get("id", model.get("name", "Unknown"))
                            print(f"      {j}. {model_name}")
                        if len(output_data["models"]) > 5:
                            print(
                                f"      ... and {len(output_data['models']) - 5} more models"
                            )
                    else:
                        print(f"   Output: {str(output_data)[:200]}...")
            except Exception as e:
                print(f"   Could not parse tool data: {e}")
    print(f"   Total time: {mcp_calls['total_time']:.2f} seconds")
    print(f"   Box MCP calls: {len(mcp_calls['mcp_calls_performed'])}")

# Examples

**Note:** Some queries may consume more tokens than others depending on the amount of tool calls the model makes. Please be aware of various rate limits that are tied to your API keys if you happen to run into any rate limit errors. 

---

## Demo 1: Let's query a LLM + Box MCP server on AI startup fundraising in SF.

In [ ]:
from IPython.display import Markdown

# NOTE - we say "based on private data in my Box account" just to make it more likely that the MCP server will be used
query = "Summarize the state of AI inference provider (e.g., Groq, Cerebras, SambaNova, etc.) startup fundraising in 2025 based on private data in my Box account"

some_interesting_private_content = connect_groq_to_box(
    client,
    tools,
    query,
)

Let's display the agent's response in markdown format.

In [ ]:
some_interesting_private_content

In [ ]:
Markdown(some_interesting_private_content["content"])

Let's examine the agent's intermediate steps, including how it calls different tools and configures tool arguments such as `search_depth`, `time_range`, `max_results`, and more.

In [ ]:
print_mcp_calls(some_interesting_private_content)

## Demo 1.1: Try the same prompt without Box MCP (LLM only)
Notice, the LLM will hallucinate fundraising data without access to our private data in Box.

In [ ]:

query = "Summarize the state of AI inference provider (e.g., Groq, Cerebras, SambaNova, etc.) startup fundraising in 2025"

some_interesting_private_content = connect_groq_to_box(
    client,
    [], # NOTE - no tools this time
    query,
)
Markdown(some_interesting_private_content["content"])

## Demo 2: Try it Yourself

Now it's your turn! Replace the query with something your interested in from your private data in Box.

In [ ]:
your_query = "Your Query Here"  # Change this!

custom_response = connect_groq_to_box(client, tools, your_query)

In [ ]:
Markdown(custom_response["content"])

In [ ]:
print_mcp_calls(custom_response)